# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.
    
    

In [2]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-28dde04e63-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-02.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
       results = sparql.query()
       json_results = results.convert()
       if len(json_results['results']['bindings'])==0:
          print("Empty")
          return 0
    
       for bindings in json_results['results']['bindings']:
          print( [ (var, value['value'])  for var, value in bindings.items() ] )

       return len(json_results['results']['bindings'])

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# Movie Workflow Series ("Horror Franchises explorative search") 

Consider the following exploratory scenario:


> we are interested in horror movies, in particular we care about the slasher subgenre, and we want to compare the Halloween and the Friday the 13th francise. 

Background knowledge useful for better conducting the search: Halloween is a movie from 1978 with many sequels and the set of Halloween + all the sequels is called the "Halloween franchise". The same considerations hold for Friday the 13th.


## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | profession    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wd:Q221103`     | Halloween       | node |
| `wd:Q1243029`| Friday the 13th       | node |





Also consider

```
wd:Q1243029 ?p ?obj .
```

is the BGP to retrieve all **properties of Friday the 13th**

The workload should


1. Investigate the workers of the two original movies and check any commonality 

2. Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)

3. Check which actor/actress did more movies after the release of these two films (consider the films separately)

4. Get all the movies of the two franchises (return year and director)

5. Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)? 

6. Investigate which franchise is highly regarded by the critics overall

7. Investigate which franchise costed more 

### Task 1

In [9]:
# start your workflow here

In [10]:
queryString = """
SELECT *
WHERE { 

wd:Q1243029 ?p ?obj .
} 
"""

print("Results")
run_query(queryString)

Results
[('p', 'http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), ('obj', 'http://wikiba.se/ontology#Item')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P244'), ('obj', 'https://id.loc.gov/authorities/names/no98033420')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P3212'), ('obj', 'urn:isan:0000-0000-31EA-0000-G-0000-0000-Q')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P4276'), ('obj', 'https://data.cinematheque.qc.ca/data/Work1467')]
[('p', 'http://www.wikidata.org/prop/direct-normalized/P646'), ('obj', 'http://g.co/kg/m/016mng')]
[('p', 'http://www.wikidata.org/prop/direct/P1237'), ('obj', 'friday13th')]
[('p', 'http://www.wikidata.org/prop/direct/P1258'), ('obj', 'm/friday_the_13th_part_1')]
[('p', 'http://www.wikidata.org/prop/direct/P1265'), ('obj', '505')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('obj', 'http://www.wikidata.org/entity/Q1067324')]
[('p', 'http://www.wikidata.org/prop/direct/P136'), ('obj', 'http://www.wikidata.org/ent

118

In [11]:
# Return all properties outgoing from Halloween film
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q221103 ?property ?object.
    ?property <http://schema.org/name> ?propertyName.
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P3593'), ('propertyName', 'AFI Catalog of Feature Films ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1562'), ('propertyName', 'AllMovie title ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2465'), ('propertyName', 'Allcinema film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1265'), ('propertyName', 'AlloCiné film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9586'), ('propertyName', 'Apple TV movie ID')]
[('property', 'http://www.wikidata.org/prop/direct/P4632'), ('propertyName', 'Bechdel Test Movie List ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1237'), ('propertyName', 'Box Office Mojo film ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P4665'), ('propertyName', 'CineMagia title ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3933'), ('propertyName', 'Cinema.de ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3077'), (

100

In [12]:
# Return all properties ingoin from Halloween film
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    ?object ?property wd:Q221103.
    ?property <http://schema.org/name> ?propertyName.
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P144'), ('propertyName', 'based on')]
[('property', 'http://www.wikidata.org/prop/direct/P4584'), ('propertyName', 'first appearance')]
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('propertyName', 'follows')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('propertyName', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P840'), ('propertyName', 'narrative location')]
[('property', 'http://www.wikidata.org/prop/direct/P800'), ('propertyName', 'notable work')]
[('property', 'http://www.wikidata.org/prop/direct/P1441'), ('propertyName', 'present in work')]
[('property', 'http://www.wikidata.org/prop/direct/P1056'), ('propertyName', 'product or material produced')]
[('property', 'http://www.wikidata.org/prop/direct/P6166'), ('propertyName', 'quotes work')]


9

In [13]:
# Return all workers related to the Halloween film by the following relation
# "performer"
queryString = """
SELECT DISTINCT ?performer ?performerName
WHERE { 

    wd:Q1243029 ?property ?performer.
    ?performer <http://schema.org/name> ?performerName.
    
    FILTER(?property = wdt:P175)
} 
ORDER BY ?performer
"""

print("Results")
run_query(queryString)

Results
[('performer', 'http://www.wikidata.org/entity/Q664025'), ('performerName', 'Harry Manfredini')]


1

In [14]:
# Return all workers related to the Friday the 13th film by the following relation
# "performer"
queryString = """
SELECT DISTINCT ?performer ?performerName
WHERE { 

    wd:Q1243029 ?property ?performer.
    ?performer <http://schema.org/name> ?performerName.
    
    FILTER(?property = wdt:P175)
} 
ORDER BY ?performer
"""

print("Results")
run_query(queryString)

Results
[('performer', 'http://www.wikidata.org/entity/Q664025'), ('performerName', 'Harry Manfredini')]


1

In [15]:
# Return all outgoing properties from Halloween and Friday the 13th films
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    {
        wd:Q221103 ?property ?object.
        ?property <http://schema.org/name> ?propertyName.
    }
    UNION
    {
        wd:Q1243029 ?property ?object.
        ?property <http://schema.org/name> ?propertyName.
    }
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P3593'), ('propertyName', 'AFI Catalog of Feature Films ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1562'), ('propertyName', 'AllMovie title ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2465'), ('propertyName', 'Allcinema film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1265'), ('propertyName', 'AlloCiné film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9586'), ('propertyName', 'Apple TV movie ID')]
[('property', 'http://www.wikidata.org/prop/direct/P4632'), ('propertyName', 'Bechdel Test Movie List ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1237'), ('propertyName', 'Box Office Mojo film ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P2758'), ('propertyName', 'CNC film rating (France)')]
[('property', 'http://www.wikidata.org/prop/direct/P4665'), ('propertyName', 'CineMagia title ID')]
[('property', 'http://www.wikidata.org/prop/direc

110

In [16]:
# Return all ingoing properties from Halloween and Friday the 13th films
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    {
        ?object ?property wd:Q221103.
        ?property <http://schema.org/name> ?propertyName.
    }
    UNION
    {
        ?object ?property wd:Q1243029.
        ?property <http://schema.org/name> ?propertyName.
    }
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P144'), ('propertyName', 'based on')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('propertyName', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P4584'), ('propertyName', 'first appearance')]
[('property', 'http://www.wikidata.org/prop/direct/P155'), ('propertyName', 'follows')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('propertyName', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P840'), ('propertyName', 'narrative location')]
[('property', 'http://www.wikidata.org/prop/direct/P800'), ('propertyName', 'notable work')]
[('property', 'http://www.wikidata.org/prop/direct/P1441'), ('propertyName', 'present in work')]
[('property', 'http://www.wikidata.org/prop/direct/P1056'), ('propertyName', 'product or material produced')]
[('property', 'http://www.wikidata.org/prop/direct/P6166'), ('propertyName', 'quotes work')]


10

In [17]:
# Return people that worked for Halloween with the following relations
# "cast member", "director", "director of photography", "performer", "producer", "production designer",
# "screenwriter", "composer", "executive prodicer"
queryString = """
SELECT DISTINCT ?workerName (GROUP_CONCAT(?propertyName; separator=" ,") AS ?role) 
WHERE { 

    wd:Q221103 ?property ?worker.
    
    ?worker <http://schema.org/name> ?workerName.
    
    ?property <http://schema.org/name> ?propertyName.
    FILTER(?property = wdt:P161 || ?property = wdt:P57 || ?property = wdt:P344 || ?property = wdt:P175 
    || ?property = wdt:P162 || ?property = wdt:2554 || ?property = wdt:P58 || ?property = wdt:P86 || ?property = wdt:P1431)

} 
GROUP BY ?workerName
ORDER BY ?workerName
"""

print("Results")
run_query(queryString)

Results
[('workerName', 'Arthur Malet'), ('role', 'cast member')]
[('workerName', 'Charles Cyphers'), ('role', 'cast member')]
[('workerName', 'Dean Cundey'), ('role', 'director of photography')]
[('workerName', 'Debra Hill'), ('role', 'producer ,screenwriter')]
[('workerName', 'Donald Pleasence'), ('role', 'cast member')]
[('workerName', 'Gabriel Cattand'), ('role', 'cast member')]
[('workerName', 'Irwin Yablans'), ('role', 'producer')]
[('workerName', 'Jamie Lee Curtis'), ('role', 'cast member')]
[('workerName', 'John Carpenter'), ('role', 'cast member ,performer ,director ,screenwriter ,composer')]
[('workerName', 'John Michael Graham'), ('role', 'cast member')]
[('workerName', 'Kyle Richards'), ('role', 'cast member')]
[('workerName', 'Moustapha Akkad'), ('role', 'producer ,executive producer')]
[('workerName', 'Nancy Kyes'), ('role', 'cast member')]
[('workerName', 'Nancy Stephens'), ('role', 'cast member')]
[('workerName', 'Nick Castle'), ('role', 'cast member')]
[('workerName', 

17

In [18]:
# Return people that worked for Friday the 13th with the following relations
# "cast member", "director", "director of photography", "performer", "producer", "production designer",
# "screenwriter", "composer", "executive prodicer"
queryString = """
SELECT DISTINCT ?workerName (GROUP_CONCAT(?propertyName; separator=" ,") AS ?role) 
WHERE { 

    wd:Q1243029 ?property ?worker.
    
    ?worker <http://schema.org/name> ?workerName.
    
    ?property <http://schema.org/name> ?propertyName.
    FILTER(?property = wdt:P161 || ?property = wdt:P57 || ?property = wdt:P344 || ?property = wdt:P175 
    || ?property = wdt:P162 || ?property = wdt:2554 || ?property = wdt:P58 || ?property = wdt:P86 || ?property = wdt:P1431)

} 
GROUP BY ?workerName
ORDER BY ?workerName
"""

print("Results")
run_query(queryString)

Results
[('workerName', 'Adrienne King'), ('role', 'cast member')]
[('workerName', 'Ari Lehman'), ('role', 'cast member')]
[('workerName', 'Barry Abrams'), ('role', 'director of photography')]
[('workerName', 'Betsy Palmer'), ('role', 'cast member')]
[('workerName', 'Harry Crosby'), ('role', 'cast member')]
[('workerName', 'Harry Manfredini'), ('role', 'performer ,composer')]
[('workerName', 'Jeannine Taylor'), ('role', 'cast member')]
[('workerName', 'Kevin Bacon'), ('role', 'cast member')]
[('workerName', 'Laurie Bartram'), ('role', 'cast member')]
[('workerName', 'Rex Everhart'), ('role', 'cast member')]
[('workerName', 'Robbi Morgan'), ('role', 'cast member')]
[('workerName', 'Sean S. Cunningham'), ('role', 'producer ,director')]
[('workerName', 'Victor Miller'), ('role', 'screenwriter')]


13

In [62]:
# Investigate the workers of the two original movies and check any commonality
queryString = """
SELECT ?workerName (GROUP_CONCAT(DISTINCT ?filmTitle; separator=" ,") AS ?workedIn) (GROUP_CONCAT(?propertyName; separator=" ,") AS ?role)
WHERE { 

    {
        wd:Q221103 ?property ?worker.
        wd:Q221103 wdt:P1476 ?filmTitle.
    }
    UNION
    {
        wd:Q1243029 ?property ?worker.
        wd:Q1243029 wdt:P1476 ?filmTitle.
    }
    
    ?worker <http://schema.org/name> ?workerName.
    
    ?property <http://schema.org/name> ?propertyName.
    FILTER(?property = wdt:P161 || ?property = wdt:P57 || ?property = wdt:P344 || ?property = wdt:P175 
    || ?property = wdt:P162 || ?property = wdt:2554 || ?property = wdt:P58 || ?property = wdt:P86 || ?property = wdt:P1431)

} 
GROUP BY ?workerName
ORDER BY ?workerName
"""

print("Results")
run_query(queryString)

Results
[('workerName', 'Adrienne King'), ('workedIn', 'Friday the 13th'), ('role', 'cast member')]
[('workerName', 'Ari Lehman'), ('workedIn', 'Friday the 13th'), ('role', 'cast member')]
[('workerName', 'Arthur Malet'), ('workedIn', 'Halloween'), ('role', 'cast member')]
[('workerName', 'Barry Abrams'), ('workedIn', 'Friday the 13th'), ('role', 'director of photography')]
[('workerName', 'Betsy Palmer'), ('workedIn', 'Friday the 13th'), ('role', 'cast member')]
[('workerName', 'Charles Cyphers'), ('workedIn', 'Halloween'), ('role', 'cast member')]
[('workerName', 'Dean Cundey'), ('workedIn', 'Halloween'), ('role', 'director of photography')]
[('workerName', 'Debra Hill'), ('workedIn', 'Halloween'), ('role', 'producer ,screenwriter')]
[('workerName', 'Donald Pleasence'), ('workedIn', 'Halloween'), ('role', 'cast member')]
[('workerName', 'Gabriel Cattand'), ('workedIn', 'Halloween'), ('role', 'cast member')]
[('workerName', 'Harry Crosby'), ('workedIn', 'Friday the 13th'), ('role', 'c

30

### Point 1 Finish

DESCRIPTION: I started by checking all ingoing and outgoing properties from both the given films.
From that I decided to keep all the following relations: "cast member" wdt:P161, "director" wdt:P57, "director of photography"wdt:P344, "performer" wdt:P175, "producer" wdt:P162, "production designer" wdt:2554, "screenwriter" wdt:P58, "composer" wdt:P86, "executive producer" wdt:P1431. All these relations connect the film to the worker.


RESULT: The result shows the name of all the people that have worked for both films along with their role. We can observe that “John Carpenter” is the one with the most roles.

### Task 2

In [20]:
# Return all distinct professions of cast member in both films
queryString = """
SELECT DISTINCT ?profession ?professionName
WHERE { 

    {
        wd:Q221103 wdt:P161 ?worker.
    }
    UNION
    {
        wd:Q1243029 wdt:P161 ?worker.
    }
    
    
    ?worker <http://schema.org/name> ?workerName.
    ?worker wdt:P106 ?profession.
    ?profession <http://schema.org/name> ?professionName.

} 
ORDER BY ?professionName
"""

print("Results")
run_query(queryString)

Results
[('profession', 'http://www.wikidata.org/entity/Q33999'), ('professionName', 'actor')]
[('profession', 'http://www.wikidata.org/entity/Q805221'), ('professionName', 'ballet dancer')]
[('profession', 'http://www.wikidata.org/entity/Q806798'), ('professionName', 'banker')]
[('profession', 'http://www.wikidata.org/entity/Q948329'), ('professionName', 'character actor')]
[('profession', 'http://www.wikidata.org/entity/Q4853732'), ('professionName', "children's writer")]
[('profession', 'http://www.wikidata.org/entity/Q36834'), ('professionName', 'composer')]
[('profession', 'http://www.wikidata.org/entity/Q5716684'), ('professionName', 'dancer')]
[('profession', 'http://www.wikidata.org/entity/Q3455803'), ('professionName', 'director')]
[('profession', 'http://www.wikidata.org/entity/Q11481802'), ('professionName', 'dub actor')]
[('profession', 'http://www.wikidata.org/entity/Q10800557'), ('professionName', 'film actor')]
[('profession', 'http://www.wikidata.org/entity/Q2526255'), 

26

In [21]:
# Return all distinct properties of cast member in both films
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    {
        wd:Q221103 wdt:P161 ?castMember.
    }
    UNION
    {
        wd:Q1243029 wdt:P161 ?castMember.
    }
    
    
    ?castMember ?property ?object.
    ?property <http://schema.org/name> ?propertyName.

} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P7119'), ('propertyName', 'AMPAS collections person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7109'), ('propertyName', 'Acharts.co artist ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8895'), ('propertyName', 'All the Tropes identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P2019'), ('propertyName', 'AllMovie person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1728'), ('propertyName', 'AllMusic artist ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7214'), ('propertyName', 'Allcinema person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1266'), ('propertyName', 'AlloCiné person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5340'), ('propertyName', 'American Film Institute person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2850'), ('propertyName', 'Apple Music artist ID (American version)')]
[('property', 'http://www.wikidata.o

232

In [22]:
# Return all distinct filmography of cast member in both films
queryString = """
SELECT DISTINCT ?castMemberName ?filmographyName
WHERE { 

    {
        wd:Q221103 wdt:P161 ?castMember.
    }
    UNION
    {
        wd:Q1243029 wdt:P161 ?castMember.
    }
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?castMember wdt:P1283 ?filmography.
    ?filmography <http://schema.org/name> ?filmographyName.

} 
ORDER BY ?castMemberName
"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'John Carpenter'), ('filmographyName', 'John Carpenter filmography')]
[('castMemberName', 'Kevin Bacon'), ('filmographyName', 'Kevin Bacon filmography')]


2

In [23]:
# Return all distinct filmography of cast member in both films
queryString = """
SELECT DISTINCT ?castMemberName ?filmographyName
WHERE { 

    {
        wd:Q221103 wdt:P161 ?castMember.
    }
    UNION
    {
        wd:Q1243029 wdt:P161 ?castMember.
    }
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?castMember wdt:P1283 ?filmography.
    ?filmography <http://schema.org/name> ?filmographyName.

} 
ORDER BY ?castMemberName
"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'John Carpenter'), ('filmographyName', 'John Carpenter filmography')]
[('castMemberName', 'Kevin Bacon'), ('filmographyName', 'Kevin Bacon filmography')]


2

In [24]:
# Return all properties of people that are actor
# "actor", "character actor", "film actor", "stage actor", "television actor"
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    ?actor wdt:P106 ?profession.
    
    FILTER(?profession = wd:Q33999 || ?profession = wd:Q948329 || ?profession = wd:Q10800557 || ?profession = wd:Q2259451
    || ?profession = wd:Q10798782)
    
    ?actor ?property ?object.
    ?property <http://schema.org/name> ?propertyName.

} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P8992'), ('propertyName', '1914-1918-Online ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5716'), ('propertyName', '2006 Commonwealth Games athlete ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7397'), ('propertyName', '247Sports ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3314'), ('propertyName', '365chess player ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9046'), ('propertyName', 'A*dS Encyclopedia ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9634'), ('propertyName', 'AADFI member ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8783'), ('propertyName', 'AAGM person or institution ID')]
[('property', 'http://www.wikidata.org/prop/direct/P3525'), ('propertyName', 'ACB.com player ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2179'), ('propertyName', 'ACM Classification Code (2012)')]
[('property', 'http://www.wikidata.org/prop/direct/P864'), (

2496

In [25]:
# Return all film of every actor that have acted in Halloween
queryString = """
SELECT DISTINCT ?castMemberName
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?castMember wdt:P161 ?film.
    ?film wdt:P1476 ?filmTitle.
    
} 
ORDER BY ?castMemberName

"""

print("Results")
run_query(queryString)

Results
Empty


0

In [26]:
# Return all outgoing properties of every actor that have acted in Halloween
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?castMember ?property ?object.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName

"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P7119'), ('propertyName', 'AMPAS collections person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7109'), ('propertyName', 'Acharts.co artist ID')]
[('property', 'http://www.wikidata.org/prop/direct/P8895'), ('propertyName', 'All the Tropes identifier')]
[('property', 'http://www.wikidata.org/prop/direct/P2019'), ('propertyName', 'AllMovie person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1728'), ('propertyName', 'AllMusic artist ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7214'), ('propertyName', 'Allcinema person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1266'), ('propertyName', 'AlloCiné person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P5340'), ('propertyName', 'American Film Institute person ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2850'), ('propertyName', 'Apple Music artist ID (American version)')]
[('property', 'http://www.wikidata.o

219

In [27]:
# Return all ingoing properties of every actor that have acted in Halloween
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?object ?property ?castMember.
    ?property <http://schema.org/name> ?propertyName.
    
} 
ORDER BY ?propertyName

"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P1877'), ('propertyName', 'after a work by')]
[('property', 'http://www.wikidata.org/prop/direct/P161'), ('propertyName', 'cast member')]
[('property', 'http://www.wikidata.org/prop/direct/P971'), ('propertyName', 'category combines topics')]
[('property', 'http://www.wikidata.org/prop/direct/P301'), ('propertyName', "category's main topic")]
[('property', 'http://www.wikidata.org/prop/direct/P40'), ('propertyName', 'child')]
[('property', 'http://www.wikidata.org/prop/direct/P86'), ('propertyName', 'composer')]
[('property', 'http://www.wikidata.org/prop/direct/P170'), ('propertyName', 'creator')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('propertyName', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P57'), ('propertyName', 'director')]
[('property', 'http://www.wikidata.org/prop/direct/P344'), ('propertyName', 'director of photography')]
[('property', 'http://www.wikidata.org/prop/dir

23

In [28]:
# Return all films of every actor that have acted in Halloween
queryString = """
SELECT DISTINCT ?castMemberName ?filmTitle
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?film wdt:P161 ?castMember.
    ?film wdt:P1476 ?filmTitle.
    
} 
ORDER BY ?castMemberName

"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Vanishing Point')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Oh, God! You Devil')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Hook')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'The Waltons')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Mary Poppins')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Dallas')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'In the Heat of the Night')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Heaven Can Wait')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Wonder Woman')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Young Frankenstein')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Lt. Robin Crusoe, U.S.N.')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'King Rat')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'MacGyver')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Dick Tracy')]
[('castMemberName', 'Ar

271

In [29]:
# Return all awards of every actor that have acted in Halloween
queryString = """
SELECT DISTINCT ?castMemberName ?awardName
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?castMember wdt:P166 ?award.
    ?award <http://schema.org/name> ?awardName.
    
} 
ORDER BY ?castMemberName

"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'Donald Pleasence'), ('awardName', 'Officer of the Order of the British Empire')]
[('castMemberName', 'Jamie Lee Curtis'), ('awardName', 'Golden Globe Award for Best Actress – Motion Picture Musical or Comedy')]
[('castMemberName', 'Jamie Lee Curtis'), ('awardName', 'star on Hollywood Walk of Fame')]
[('castMemberName', 'Jamie Lee Curtis'), ('awardName', 'BAFTA Award for Best Actress in a Supporting Role')]


4

In [30]:
# Return all awards of every actor that have acted in Halloween
queryString = """
SELECT DISTINCT ?castMemberName ?awardName
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?award wdt:P1346 ?castMember.
    ?award <http://schema.org/name> ?awardName.
    
} 
ORDER BY ?castMemberName

"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'Donald Pleasence'), ('awardName', 'British Academy Television Award for Best Actor')]
[('castMemberName', 'Jamie Lee Curtis'), ('awardName', 'Saturn Award for Best Actress')]
[('castMemberName', 'Jamie Lee Curtis'), ('awardName', 'BAFTA Award for Best Actress in a Supporting Role')]


3

In [31]:
# Return the publication dates of both films
queryString = """
SELECT DISTINCT ?title ?publicationDate
WHERE { 

    {
        wd:Q221103 wdt:P577 ?publicationDate.
        wd:Q221103 wdt:P1476 ?title
    }
    UNION
    {
        wd:Q1243029 wdt:P577 ?publicationDate.
        wd:Q1243029 wdt:P1476 ?title
    }
    
} 

"""

print("Results")
run_query(queryString)

Results
[('title', 'Halloween'), ('publicationDate', '1978-10-25T00:00:00Z')]
[('title', 'Halloween'), ('publicationDate', '1979-07-06T00:00:00Z')]
[('title', 'Halloween'), ('publicationDate', '1979-08-06T00:00:00Z')]
[('title', 'Friday the 13th'), ('publicationDate', '1980-05-09T00:00:00Z')]
[('title', 'Friday the 13th'), ('publicationDate', '1980-10-23T00:00:00Z')]
[('title', 'Friday the 13th'), ('publicationDate', '1980-11-10T00:00:00Z')]


6

In [32]:
# Return the most famous actor that acted in Halloween (consider both number of films and awards)
queryString = """
SELECT ?castMemberName (COUNT(?filmOrAward) AS ?fameScore)
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    {
        ?filmOrAward wdt:P161 ?castMember.
    }
    UNION
    {
        ?castMember wdt:P166 ?filmOrAward.
    }
    UNION
    {
        ?filmOrAward wdt:P1346 ?castMember.
    }
    
}
GROUP BY ?castMemberName
ORDER BY DESC (?fameScore)
LIMIT 1

"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'Donald Pleasence'), ('fameScore', '130')]


1

In [33]:
# Return all films of every actor that have acted in Halloween at the time of the release

queryString = """
SELECT DISTINCT ?castMemberName ?filmTitle ?filmPublicationDate
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.
    
    ?castMember <http://schema.org/name> ?castMemberName.
    
    ?film wdt:P161 ?castMember.
    ?film wdt:P1476 ?filmTitle.
    ?film wdt:P577 ?filmPublicationDate.
    
    FILTER(?filmPublicationDate < '1978-10-25T00:00:00Z'^^xsd:dateTime)
} 
ORDER BY ?castMemberName

"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Vanishing Point'), ('filmPublicationDate', '1971-01-01T00:00:00Z')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Mary Poppins'), ('filmPublicationDate', '1964-10-01T00:00:00Z')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Mary Poppins'), ('filmPublicationDate', '1965-08-23T00:00:00Z')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Mary Poppins'), ('filmPublicationDate', '1965-10-22T00:00:00Z')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'In the Heat of the Night'), ('filmPublicationDate', '1967-08-02T00:00:00Z')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Heaven Can Wait'), ('filmPublicationDate', '1978-06-28T00:00:00Z')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Young Frankenstein'), ('filmPublicationDate', '1974-12-15T00:00:00Z')]
[('castMemberName', 'Arthur Malet'), ('filmTitle', 'Young Frankenstein'), ('filmPublicationDate', '1975-09-05T00:00:00Z')]
[('castMemberName', 'Arthu

136

In [63]:
# Check who was the most famous actor/actress on Halloween and Friday the 13th at the time of the release
# (e.g., the actor/actress who acted in more movies before and/or the actor/actress who won more academy awards)

queryString = """
SELECT ?filmTitle ?castMemberName ?fameScore
WHERE {

    {
        SELECT DISTINCT ?castMemberName (COUNT(DISTINCT ?film) AS ?fameScore) ?filmTitle
        WHERE { 

            wd:Q221103 wdt:P161 ?castMember.
            wd:Q221103 wdt:P1476 ?filmTitle.

            ?castMember <http://schema.org/name> ?castMemberName.

            ?film wdt:P161 ?castMember.
            ?film wdt:P577 ?filmPublicationDate.
            FILTER(?filmPublicationDate < '1978-10-25T00:00:00Z'^^xsd:dateTime)
        } 
        ORDER BY DESC (?fameScore)
        LIMIT 1
    }
    UNION
    {
        SELECT DISTINCT ?castMemberName (COUNT(DISTINCT ?film) AS ?fameScore) ?filmTitle
        WHERE { 

            wd:Q1243029 wdt:P161 ?castMember.
            wd:Q1243029 wdt:P1476 ?filmTitle.

            ?castMember <http://schema.org/name> ?castMemberName.

            ?film wdt:P161 ?castMember.
            ?film wdt:P577 ?filmPublicationDate.
            FILTER(?filmPublicationDate < '1980-05-09T00:00:00Z'^^xsd:dateTime)
        } 
        ORDER BY DESC (?fameScore)
        LIMIT 1
    }

}


"""

print("Results")
run_query(queryString)

Results
[('filmTitle', 'Halloween'), ('castMemberName', 'Donald Pleasence'), ('fameScore', '74')]
[('filmTitle', 'Friday the 13th'), ('castMemberName', 'Betsy Palmer'), ('fameScore', '6')]


2

### Piont 2 Finish

DESCRIPTION: First I use relation “professions” wdt:P161 to find the “actor” class.
Then, I started again by taking a general person that as profession is: "actor" wd:Q33999, "character actor" wd:Q948329, "film actor" wd:Q10800557, "stage actor" wd:Q2259451, "television actor" wd:Q10798782 and return all its properties.
Then I tried a different approach and returned all the ingoing and outgoing properties from the cast members of Halloween and I found the “winner” wdt:P1346 property which connects an award to a person.
Then I searched for the release dates of both films.
I try to find a possible way to compute the fame of an actor. I decided to count the number of films and receiver awards. In the end I realize that I was not able to find the date of an award, so I decided to use only the number of films.


RESULT: The result shows the most famous actors for the films. The “fameScore” was computed by counting the number of films that they made before the release of Halloween and Friday 13th.

### Task 3 Start

In [35]:
# Get the film done after Halloween by all cast members

queryString = """
SELECT DISTINCT ?castMemberName (COUNT(DISTINCT ?film) AS ?filmAfterRelease)
WHERE { 

    wd:Q221103 wdt:P161 ?castMember.

    ?castMember <http://schema.org/name> ?castMemberName.

    ?film wdt:P161 ?castMember.
    ?film wdt:P577 ?filmPublicationDate.
    FILTER(?filmPublicationDate > '1978-10-25T00:00:00Z'^^xsd:dateTime)
} 
ORDER BY DESC (?filmAfterRelease)
  
"""

print("Results")
run_query(queryString)

Results
[('castMemberName', 'Donald Pleasence'), ('filmAfterRelease', '55')]
[('castMemberName', 'Jamie Lee Curtis'), ('filmAfterRelease', '52')]
[('castMemberName', 'P. J. Soles'), ('filmAfterRelease', '20')]
[('castMemberName', 'John Carpenter'), ('filmAfterRelease', '18')]
[('castMemberName', 'Gabriel Cattand'), ('filmAfterRelease', '12')]
[('castMemberName', 'Arthur Malet'), ('filmAfterRelease', '10')]
[('castMemberName', 'Charles Cyphers'), ('filmAfterRelease', '9')]
[('castMemberName', 'Kyle Richards'), ('filmAfterRelease', '8')]
[('castMemberName', 'Nick Castle'), ('filmAfterRelease', '6')]
[('castMemberName', 'Nancy Kyes'), ('filmAfterRelease', '5')]
[('castMemberName', 'Nancy Stephens'), ('filmAfterRelease', '4')]
[('castMemberName', 'Tony Moran'), ('filmAfterRelease', '3')]
[('castMemberName', 'John Michael Graham'), ('filmAfterRelease', '1')]


13

In [64]:
# Check which actor/actress did more movies after the release of these two films (consider the films separately)

queryString = """
SELECT ?filmTitle ?castMemberName ?filmAfterRelease
WHERE {

    {
        SELECT DISTINCT ?castMemberName (COUNT(DISTINCT ?film) AS ?filmAfterRelease) ?filmTitle
        WHERE { 

            wd:Q221103 wdt:P161 ?castMember.
            wd:Q221103 wdt:P1476 ?filmTitle.

            ?castMember <http://schema.org/name> ?castMemberName.

            ?film wdt:P161 ?castMember.
            ?film wdt:P577 ?filmPublicationDate.
            FILTER(?filmPublicationDate > '1978-10-25T00:00:00Z'^^xsd:dateTime)
        } 
        ORDER BY DESC (?filmAfterRelease)
        LIMIT 1
    }
    UNION
    {
        SELECT DISTINCT ?castMemberName (COUNT(DISTINCT ?film) AS ?filmAfterRelease) ?filmTitle
        WHERE { 

            wd:Q1243029 wdt:P161 ?castMember.
            wd:Q1243029 wdt:P1476 ?filmTitle.

            ?castMember <http://schema.org/name> ?castMemberName.

            ?film wdt:P161 ?castMember.
            ?film wdt:P577 ?filmPublicationDate.
            FILTER(?filmPublicationDate > '1980-05-09T00:00:00Z'^^xsd:dateTime)
        } 
        ORDER BY DESC (?filmAfterRelease)
        LIMIT 1
    }

}


"""

print("Results")
run_query(queryString)

Results
[('filmTitle', 'Halloween'), ('castMemberName', 'Donald Pleasence'), ('filmAfterRelease', '55')]
[('filmTitle', 'Friday the 13th'), ('castMemberName', 'Kevin Bacon'), ('filmAfterRelease', '64')]


2

### Piont 3 Finish

DESCRIPTION: This query is very similar to the previous one but with the opposite filter.

RESULT: The result shows the two actors that did more movies for both films after their release date.

### Task 4 Start

In [37]:
# Return info about "part of the series" property for Halloween node
queryString = """
SELECT DISTINCT ?object ?objectName
WHERE { 

    wd:Q221103 wdt:P179 ?object.
    ?object <http://schema.org/name> ?objectName.
} 
ORDER BY ?objectName
"""

print("Results")
run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1364022'), ('objectName', 'Halloween')]


1

In [38]:
# Return all nodes that are connected to "Halloween" serie  
queryString = """
SELECT DISTINCT ?film ?filmTitle
WHERE { 

    ?film wdt:P179 wd:Q1364022.
    ?film wdt:P1476 ?filmTitle.
} 
ORDER BY ?filmTitle
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('filmTitle', 'Halloween')]
[('film', 'http://www.wikidata.org/entity/Q909063'), ('filmTitle', 'Halloween')]
[('film', 'http://www.wikidata.org/entity/Q959853'), ('filmTitle', 'Halloween 4: The Return of Michael Myers')]
[('film', 'http://www.wikidata.org/entity/Q976728'), ('filmTitle', 'Halloween 5: The Revenge of Michael Myers')]
[('film', 'http://www.wikidata.org/entity/Q1050815'), ('filmTitle', 'Halloween H20: 20 Years Later')]
[('film', 'http://www.wikidata.org/entity/Q850312'), ('filmTitle', 'Halloween II')]
[('film', 'http://www.wikidata.org/entity/Q908690'), ('filmTitle', 'Halloween II')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('filmTitle', 'Halloween III: Season of the Witch')]
[('film', 'http://www.wikidata.org/entity/Q176488'), ('filmTitle', 'Halloween: Resurrection')]
[('film', 'http://www.wikidata.org/entity/Q1138055'), ('filmTitle', 'Halloween: The Curse of Michael Myers')]


10

In [39]:
# Return all films of Halloween franchise with year
queryString = """
SELECT DISTINCT ?film ?filmTitle (MIN(?outYear) AS ?year)
WHERE { 

    ?film wdt:P179 wd:Q1364022.
    ?film wdt:P1476 ?filmTitle.
    ?film wdt:P577 ?publicationDate.
    BIND(YEAR(?publicationDate) AS ?outYear)
} 
ORDER BY (?year)
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('filmTitle', 'Halloween'), ('year', '1978')]
[('film', 'http://www.wikidata.org/entity/Q850312'), ('filmTitle', 'Halloween II'), ('year', '1981')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('filmTitle', 'Halloween III: Season of the Witch'), ('year', '1982')]
[('film', 'http://www.wikidata.org/entity/Q959853'), ('filmTitle', 'Halloween 4: The Return of Michael Myers'), ('year', '1988')]
[('film', 'http://www.wikidata.org/entity/Q976728'), ('filmTitle', 'Halloween 5: The Revenge of Michael Myers'), ('year', '1989')]
[('film', 'http://www.wikidata.org/entity/Q1138055'), ('filmTitle', 'Halloween: The Curse of Michael Myers'), ('year', '1995')]
[('film', 'http://www.wikidata.org/entity/Q1050815'), ('filmTitle', 'Halloween H20: 20 Years Later'), ('year', '1998')]
[('film', 'http://www.wikidata.org/entity/Q176488'), ('filmTitle', 'Halloween: Resurrection'), ('year', '2002')]
[('film', 'http://www.wikidata.org/entity/Q90

10

In [40]:
# Return all films of Halloween franchise with year and director
queryString = """
SELECT DISTINCT ?film ?filmTitle (MIN(?outYear) AS ?year) (GROUP_CONCAT(DISTINCT ?directorName; separator=" ,") AS ?directors)
WHERE { 

    ?film wdt:P179 wd:Q1364022.
    ?film wdt:P1476 ?filmTitle.
    
    ?film wdt:P57 ?director.
    ?director <http://schema.org/name> ?directorName.
    
    ?film wdt:P577 ?publicationDate.
    BIND(YEAR(?publicationDate) AS ?outYear)
} 
ORDER BY (?year)
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('filmTitle', 'Halloween'), ('year', '1978'), ('directors', 'John Carpenter')]
[('film', 'http://www.wikidata.org/entity/Q850312'), ('filmTitle', 'Halloween II'), ('year', '1981'), ('directors', 'John Carpenter ,Rick Rosenthal')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('filmTitle', 'Halloween III: Season of the Witch'), ('year', '1982'), ('directors', 'Tommy Lee Wallace')]
[('film', 'http://www.wikidata.org/entity/Q959853'), ('filmTitle', 'Halloween 4: The Return of Michael Myers'), ('year', '1988'), ('directors', 'Dwight H. Little')]
[('film', 'http://www.wikidata.org/entity/Q976728'), ('filmTitle', 'Halloween 5: The Revenge of Michael Myers'), ('year', '1989'), ('directors', 'Dominique Othenin-Girard')]
[('film', 'http://www.wikidata.org/entity/Q1138055'), ('filmTitle', 'Halloween: The Curse of Michael Myers'), ('year', '1995'), ('directors', 'Joe Chappelle')]
[('film', 'http://www.wikidata.org/entity/Q1050815

10

In [41]:
# Return info about "part of the series" property for Friday the 13th node
queryString = """
SELECT DISTINCT ?object ?objectName
WHERE { 

    wd:Q1243029 wdt:P179 ?object.
    ?object <http://schema.org/name> ?objectName.
    
} 
ORDER BY ?objectName
"""

print("Results")
run_query(queryString)

Results
[('object', 'http://www.wikidata.org/entity/Q1138878'), ('objectName', 'Friday the 13th')]


1

In [42]:
# Return all films of Friday the 13th franchise with year and director
queryString = """
SELECT DISTINCT ?film ?filmTitle (MIN(?outYear) AS ?year) (GROUP_CONCAT(DISTINCT ?directorName; separator=" ,") AS ?directors)
WHERE { 

    ?film wdt:P179 wd:Q1138878.
    ?film wdt:P1476 ?filmTitle.
    
    ?film wdt:P57 ?director.
    ?director <http://schema.org/name> ?directorName.
    
    ?film wdt:P577 ?publicationDate.
    BIND(YEAR(?publicationDate) AS ?outYear)
} 
ORDER BY (?year)
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('filmTitle', 'Friday the 13th'), ('year', '1980'), ('directors', 'Sean S. Cunningham')]
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('filmTitle', 'Friday the 13th Part 2'), ('year', '1981'), ('directors', 'Sean S. Cunningham ,Steve Miner')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('filmTitle', 'Friday the 13th Part III'), ('year', '1982'), ('directors', 'Steve Miner')]
[('film', 'http://www.wikidata.org/entity/Q1243019'), ('filmTitle', 'Friday the 13th: The Final Chapter'), ('year', '1984'), ('directors', 'Joseph Zito')]
[('film', 'http://www.wikidata.org/entity/Q286830'), ('filmTitle', 'Friday the 13th: A New Beginning'), ('year', '1985'), ('directors', 'Danny Steinmann')]
[('film', 'http://www.wikidata.org/entity/Q1454799'), ('filmTitle', 'Friday the 13th Part VI: Jason Lives'), ('year', '1986'), ('directors', 'Tom McLoughlin')]
[('film', 'http://www.wikidata.org/entity/Q1454798'), ('filmTitle', 'F

12

In [65]:
# Get all the movies of the two franchises (return year and director)
queryString = """
SELECT DISTINCT ?film ?franchiseName ?filmTitle (MIN(?outYear) AS ?year) (GROUP_CONCAT(DISTINCT ?directorName; separator=" ,") AS ?directors)
WHERE { 

    {
        ?film wdt:P179 wd:Q1364022.
        ?film wdt:P1476 ?filmTitle.
        wd:Q1364022 <http://schema.org/name> ?franchiseName.

        ?film wdt:P57 ?director.
        ?director <http://schema.org/name> ?directorName.

        ?film wdt:P577 ?publicationDate.
        BIND(YEAR(?publicationDate) AS ?outYear)
    }
    UNION
    {
        ?film wdt:P179 wd:Q1138878.
        ?film wdt:P1476 ?filmTitle.
        wd:Q1138878 <http://schema.org/name> ?franchiseName.

        ?film wdt:P57 ?director.
        ?director <http://schema.org/name> ?directorName.

        ?film wdt:P577 ?publicationDate.
        BIND(YEAR(?publicationDate) AS ?outYear)
    }
} 
ORDER BY ?franchiseName ?year
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243029'), ('franchiseName', 'Friday the 13th'), ('filmTitle', 'Friday the 13th'), ('year', '1980'), ('directors', 'Sean S. Cunningham')]
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('franchiseName', 'Friday the 13th'), ('filmTitle', 'Friday the 13th Part 2'), ('year', '1981'), ('directors', 'Sean S. Cunningham ,Steve Miner')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('franchiseName', 'Friday the 13th'), ('filmTitle', 'Friday the 13th Part III'), ('year', '1982'), ('directors', 'Steve Miner')]
[('film', 'http://www.wikidata.org/entity/Q1243019'), ('franchiseName', 'Friday the 13th'), ('filmTitle', 'Friday the 13th: The Final Chapter'), ('year', '1984'), ('directors', 'Joseph Zito')]
[('film', 'http://www.wikidata.org/entity/Q286830'), ('franchiseName', 'Friday the 13th'), ('filmTitle', 'Friday the 13th: A New Beginning'), ('year', '1985'), ('directors', 'Danny Steinmann')]
[('film', 'http://www.wikidata.org/entit

22

### Piont 4 Finish

DESCRIPTION: First I try to understand more about “part of the series” wdt:P179 property. And find the entity “Halloween” wd:Q1364022 that represents the Halloween film series. From this I returned all films that are part of it. Then from all these films I returned their year and director’s name. I proceed finding the series for “Friday the 13th “ wdt:Q1138878.
Then I combine the results and obtain what the point asks. Notice that for release date I consider the earliest one for every film.


RESULT: The result shows that Friday the 13th has twelve films while Halloween series has ten films. It is also possible to see the release date and the director’s name of every single film.

### Task 5 Start

In [44]:
# Return all workers involved in Halloween franchises with their role
queryString = """
SELECT ?workerName (GROUP_CONCAT(DISTINCT ?filmTitle; separator=" ,") AS ?workedIn) (GROUP_CONCAT(?propertyName; separator=" ,") AS ?roles)
WHERE { 


    ?film wdt:P179 wd:Q1364022.
    ?film wdt:P1476 ?filmTitle.
    wd:Q1364022 <http://schema.org/name> ?franchiseName.
    
    ?film ?property ?worker.
    ?worker <http://schema.org/name> ?workerName.
    
    ?property <http://schema.org/name> ?propertyName.
    FILTER(?property = wdt:P161 || ?property = wdt:P57 || ?property = wdt:P344 || ?property = wdt:P175 
    || ?property = wdt:P162 || ?property = wdt:2554 || ?property = wdt:P58 || ?property = wdt:P86 || ?property = wdt:P1431)



} 
GROUP BY ?workerName
ORDER BY ?workerName
"""

print("Results")
run_query(queryString)

Results
[('workerName', '"Weird Al" Yankovic'), ('workedIn', 'Halloween II'), ('roles', 'cast member')]
[('workerName', 'Adam Arkin'), ('workedIn', 'Halloween H20: 20 Years Later'), ('roles', 'cast member')]
[('workerName', 'Adam Hann-Byrd'), ('workedIn', 'Halloween H20: 20 Years Later'), ('roles', 'cast member')]
[('workerName', 'Adam Weisman'), ('workedIn', 'Halloween'), ('roles', 'cast member')]
[('workerName', 'Alan B. McElroy'), ('workedIn', 'Halloween 4: The Return of Michael Myers'), ('roles', 'screenwriter')]
[('workerName', 'Alan Howarth'), ('workedIn', 'Halloween 4: The Return of Michael Myers ,Halloween 5: The Revenge of Michael Myers ,Halloween III: Season of the Witch ,Halloween: The Curse of Michael Myers'), ('roles', 'composer ,composer ,composer ,composer')]
[('workerName', 'Ana Alicia'), ('workedIn', 'Halloween II'), ('roles', 'cast member')]
[('workerName', 'Angela Trimbur'), ('workedIn', 'Halloween II'), ('roles', 'cast member')]
[('workerName', 'Anne-Marie Martin'),

170

In [45]:
# Return the participations count of worker in Halloween franchises
queryString = """
SELECT DISTINCT ?workerName (COUNT(?propertyName) AS ?participations)
WHERE { 


    ?film wdt:P179 wd:Q1364022.
    wd:Q1364022 <http://schema.org/name> ?franchiseName.
    
    ?film ?property ?worker.
    ?worker <http://schema.org/name> ?workerName.
    
    ?property <http://schema.org/name> ?propertyName.
    FILTER(?property = wdt:P161 || ?property = wdt:P57 || ?property = wdt:P344 || ?property = wdt:P175 
    || ?property = wdt:P162 || ?property = wdt:2554 || ?property = wdt:P58 || ?property = wdt:P86 || ?property = wdt:P1431)

} 
ORDER BY DESC (?participations)

"""

print("Results")
run_query(queryString)

Results
[('workerName', 'John Carpenter'), ('participations', '13')]
[('workerName', 'Moustapha Akkad'), ('participations', '7')]
[('workerName', 'Debra Hill'), ('participations', '6')]
[('workerName', 'Rob Zombie'), ('participations', '6')]
[('workerName', 'Jamie Lee Curtis'), ('participations', '5')]
[('workerName', 'Donald Pleasence'), ('participations', '5')]
[('workerName', 'Alan Howarth'), ('participations', '4')]
[('workerName', 'Danielle Harris'), ('participations', '4')]
[('workerName', 'Paul Freeman'), ('participations', '4')]
[('workerName', 'Tommy Lee Wallace'), ('participations', '3')]
[('workerName', 'Nancy Stephens'), ('participations', '3')]
[('workerName', 'Dean Cundey'), ('participations', '3')]
[('workerName', 'Rick Rosenthal'), ('participations', '3')]
[('workerName', 'Nancy Kyes'), ('participations', '3')]
[('workerName', 'Tony Moran'), ('participations', '2')]
[('workerName', 'William Forsythe'), ('participations', '2')]
[('workerName', 'Malcolm McDowell'), ('part

170

In [66]:
# Who is most involved in a franchise (who did something in the movies, e.g., writer, director, producer)?
queryString = """
SELECT ?workerName ?franchiseName ?participations
WHERE {

    {
        SELECT ?workerName ?franchiseName (COUNT(?propertyName) AS ?participations)
        WHERE { 

            ?film wdt:P179 wd:Q1364022.
            wd:Q1364022 <http://schema.org/name> ?franchiseName.

            ?film ?property ?worker.
            ?worker <http://schema.org/name> ?workerName.

            ?property <http://schema.org/name> ?propertyName.
            FILTER(?property = wdt:P161 || ?property = wdt:P57 || ?property = wdt:P344 || ?property = wdt:P175 
            || ?property = wdt:P162 || ?property = wdt:2554 || ?property = wdt:P58 || ?property = wdt:P86 || ?property = wdt:P1431)

        } 
        ORDER BY DESC (?participations)
        LIMIT 1
    }
    UNION
     {
        SELECT ?workerName ?franchiseName (COUNT(?propertyName) AS ?participations)
        WHERE { 

            ?film wdt:P179 wd:Q1138878.
            wd:Q1138878 <http://schema.org/name> ?franchiseName.

            ?film ?property ?worker.
            ?worker <http://schema.org/name> ?workerName.

            ?property <http://schema.org/name> ?propertyName.
            FILTER(?property = wdt:P161 || ?property = wdt:P57 || ?property = wdt:P344 || ?property = wdt:P175 
            || ?property = wdt:P162 || ?property = wdt:2554 || ?property = wdt:P58 || ?property = wdt:P86 || ?property = wdt:P1431)

        } 
        ORDER BY DESC (?participations)
        LIMIT 1
    }

}



"""

print("Results")
run_query(queryString)

Results
[('workerName', 'John Carpenter'), ('franchiseName', 'Halloween'), ('participations', '13')]
[('workerName', 'Harry Manfredini'), ('franchiseName', 'Friday the 13th'), ('participations', '15')]


2

### Piont 5 Finish
DESCRIPTION: I started by considering only Halloween franchise. I consider any person involved in the series using the following properties: “cast member” wdt:P161, “director”  wdt:P57, “dir. of ph.” wdt:P344, “performer” wdt:P175, “producer” wdt:P162, “production designer” wdt:2554, “screenwriter” wdt:P58, “composer” wdt:P86, “executive producer” wdt:P1431, and I count their roles.
At the end I returned the most involved person for each film series.


RESULT: The result shows that John Carpenter was the most active person for Halloween series with 13 participations, while Harry Manfredini was the most involved for Friday the 13th with 15 participations.

### Task 6 Start

In [47]:
# Return all outgoing properies of Halloween franchise
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    wd:Q1364022 ?property ?object.
    ?property <http://schema.org/name> ?propertyName.
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P2530'), ('propertyName', 'Box Office Mojo franchise ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P373'), ('propertyName', 'Commons category')]
[('property', 'http://www.wikidata.org/prop/direct/P646'), ('propertyName', 'Freebase ID')]
[('property', 'http://www.wikidata.org/prop/direct/P7818'), ('propertyName', 'French Vikidia ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1258'), ('propertyName', 'Rotten Tomatoes ID')]
[('property', 'http://www.wikidata.org/prop/direct/P495'), ('propertyName', 'country of origin')]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('propertyName', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P136'), ('propertyName', 'genre')]
[('property', 'http://www.wikidata.org/prop/direct/P527'), ('propertyName', 'has part')]
[('property', 'http://www.wikidata.org/prop/direct/P31'), ('propertyName', 'instance of')]
[('property', 'htt

15

In [48]:
# Return all ingoing properies of Halloween franchise
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    ?object ?property wd:Q1364022.
    ?property <http://schema.org/name> ?propertyName.
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P301'), ('propertyName', "category's main topic")]
[('property', 'http://www.wikidata.org/prop/direct/P1889'), ('propertyName', 'different from')]
[('property', 'http://www.wikidata.org/prop/direct/P8345'), ('propertyName', 'media franchise')]
[('property', 'http://www.wikidata.org/prop/direct/P179'), ('propertyName', 'part of the series')]
[('property', 'http://www.wikidata.org/prop/direct/P1441'), ('propertyName', 'present in work')]
[('property', 'http://www.wikidata.org/prop/direct/P1423'), ('propertyName', 'template has topic')]


6

In [3]:
# Return the assesment of the Halloween film
queryString = """
SELECT DISTINCT ?assessment ?assessmentName
WHERE { 

    wd:Q221103 wdt:P5021 ?assessment.
    ?assessment <http://schema.org/name> ?assessmentName.
} 
"""

print("Results")
run_query(queryString)

Results
[('assessment', 'http://www.wikidata.org/entity/Q4165246'), ('assessmentName', 'Bechdel test')]
[('assessment', 'http://www.wikidata.org/entity/Q85783379'), ('assessmentName', 'Mako Mori test')]


2

In [50]:
# Return the received awards of the Halloween film
queryString = """
SELECT DISTINCT ?award ?awardName
WHERE { 

    wd:Q221103 wdt:P166 ?award.
    ?award <http://schema.org/name> ?awardName.
} 
"""

print("Results")
run_query(queryString)

Results
[('award', 'http://www.wikidata.org/entity/Q823422'), ('awardName', 'National Film Registry')]


1

In [51]:
# Return the assesment of the Friday the 13th film
queryString = """
SELECT DISTINCT ?assesment ?assesmentName
WHERE { 

    wd:Q1243029 wdt:P5021 ?assesment.
    ?assesment <http://schema.org/name> ?assesmentName.
} 
"""

print("Results")
run_query(queryString)

Results
[('assesment', 'http://www.wikidata.org/entity/Q4165246'), ('assesmentName', 'Bechdel test')]


1

In [52]:
# Return the received awards of the Friday the 13th film
queryString = """
SELECT DISTINCT ?award ?awardName
WHERE { 

    wd:Q1243029 wdt:P166 ?award.
    ?award <http://schema.org/name> ?awardName.
} 
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [53]:
# Get all the awards for all Halloween franchise films
queryString = """
SELECT DISTINCT ?film ?filmTitle (GROUP_CONCAT(?awardName; separator = " ,") AS ?awards)
WHERE { 

    ?film wdt:P179 wd:Q1364022.
    ?film wdt:P1476 ?filmTitle.
    
    ?film wdt:P166 ?award.
    ?award <http://schema.org/name> ?awardName.
} 
ORDER BY (?filmTitle)
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('filmTitle', 'Halloween'), ('awards', 'National Film Registry')]


1

In [54]:
# Get all the awards for all Friday the 13th franchise films
queryString = """
SELECT DISTINCT ?film ?filmTitle (GROUP_CONCAT(?awardName; separator = " ,") AS ?awards)
WHERE { 

    ?film wdt:P179 wd:Q1138878.
    ?film wdt:P1476 ?filmTitle.
    
    ?film wdt:P166 ?award.
    ?award <http://schema.org/name> ?awardName.
} 
ORDER BY (?filmTitle)
"""

print("Results")
run_query(queryString)

Results
Empty


0

In [72]:
# Investigate which franchise is highly regarded by the critics overall
# Not sure how to effectively evaluate the critics overall of a film
queryString = """
SELECT DISTINCT ?franchiseName (GROUP_CONCAT(?awardName; separator = " ,") AS ?awards)
WHERE { 

    {
        ?film wdt:P179 wd:Q1364022.
        wd:Q1364022 <http://schema.org/name> ?franchiseName.

        ?film wdt:P166 ?award.
        ?award <http://schema.org/name> ?awardName.
    }
    UNION
    {
        ?film wdt:P179 wd:Q1138878.
        wd:Q1138878 <http://schema.org/name> ?franchiseName.

        ?film wdt:P166 ?award.
        ?award <http://schema.org/name> ?awardName.
    }
    
} 
"""

print("Results")
run_query(queryString)

Results
[('franchiseName', 'Halloween'), ('awards', 'National Film Registry')]


1

### Point 6 Finish
DESCRIPTION: Again I started by checking all the ingoing and outgoing properties of “Halloween franchise” wd:Q1364022, without finding anything interesting. Then I searched for some relations with Halloween film computed in previous queries and I tried “assessment” wdt:P5021 relation, without any useful results. 
Then I looked for property wdt:P166 “award received” of the Halloween film. 
I performed the same reason as above for Friday the 13th film but obtaining the same conclusion.
Then I returned all awards for all the films in a franchise to tell how many awards a franchise has won.
I couldn’t find any other useful relation to evaluate the overall critics so at the end I decided to keep the number of won awards by the franchise.

RESULT: The result shows that only Halloween franchise won an award while Friday the 13th franchise has not. 

### Task 7 Start

In [55]:
# Return the cost of Halloween of film
queryString = """
SELECT DISTINCT ?cost
WHERE { 

    wd:Q221103 wdt:P2130 ?cost.
    
} 
"""

print("Results")
run_query(queryString)

Results
[('cost', '325000')]


1

In [4]:
# Return the cost of Halloween franchise films
queryString = """
SELECT ?filmTitle ?cost
WHERE { 

    ?film wdt:P179 wd:Q1364022.
    wd:Q1364022 <http://schema.org/name> ?franchiseName.
    ?film wdt:P1476 ?filmTitle.
    
    ?film wdt:P2130 ?cost
    
    
} 
"""

print("Results")
run_query(queryString)

Results
[('filmTitle', 'Halloween'), ('cost', '325000')]


1

In [57]:
# Return the properties of films of Halloween franchise
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    ?film wdt:P179 wd:Q1364022.

    ?film ?property ?object.
    ?property <http://schema.org/name> ?propertyName.
    
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P3593'), ('propertyName', 'AFI Catalog of Feature Films ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1562'), ('propertyName', 'AllMovie title ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2465'), ('propertyName', 'Allcinema film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1265'), ('propertyName', 'AlloCiné film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9586'), ('propertyName', 'Apple TV movie ID')]
[('property', 'http://www.wikidata.org/prop/direct/P4632'), ('propertyName', 'Bechdel Test Movie List ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1237'), ('propertyName', 'Box Office Mojo film ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P2758'), ('propertyName', 'CNC film rating (France)')]
[('property', 'http://www.wikidata.org/prop/direct/P4665'), ('propertyName', 'CineMagia title ID')]
[('property', 'http://www.wikidata.org/prop/direc

119

In [58]:
# Return the properties of films of Friday the 13th franchise
queryString = """
SELECT DISTINCT ?property ?propertyName
WHERE { 

    ?film wdt:P179 wd:Q1138878.

    ?film ?property ?object.
    ?property <http://schema.org/name> ?propertyName.
    
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('property', 'http://www.wikidata.org/prop/direct/P3593'), ('propertyName', 'AFI Catalog of Feature Films ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1562'), ('propertyName', 'AllMovie title ID')]
[('property', 'http://www.wikidata.org/prop/direct/P2465'), ('propertyName', 'Allcinema film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1265'), ('propertyName', 'AlloCiné film ID')]
[('property', 'http://www.wikidata.org/prop/direct/P9586'), ('propertyName', 'Apple TV movie ID')]
[('property', 'http://www.wikidata.org/prop/direct/P4632'), ('propertyName', 'Bechdel Test Movie List ID')]
[('property', 'http://www.wikidata.org/prop/direct/P1237'), ('propertyName', 'Box Office Mojo film ID (former scheme)')]
[('property', 'http://www.wikidata.org/prop/direct/P2758'), ('propertyName', 'CNC film rating (France)')]
[('property', 'http://www.wikidata.org/prop/direct/P4665'), ('propertyName', 'CineMagia title ID')]
[('property', 'http://www.wikidata.org/prop/direc

122

In [59]:
# Return the cost of films of Halloween franchise
queryString = """
SELECT DISTINCT ?film ?filmTitle ?cost
WHERE { 

    ?film wdt:P179 wd:Q1364022.
    ?film wdt:P1476 ?filmTitle.

    ?film ?property ?cost.
    FILTER(?property = wdt:P2130 || ?property = wdt:P2769)
    
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q221103'), ('filmTitle', 'Halloween'), ('cost', '325000')]
[('film', 'http://www.wikidata.org/entity/Q176488'), ('filmTitle', 'Halloween: Resurrection'), ('cost', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q909063'), ('filmTitle', 'Halloween'), ('cost', '15000000')]
[('film', 'http://www.wikidata.org/entity/Q203705'), ('filmTitle', 'Halloween III: Season of the Witch'), ('cost', '2500000')]
[('film', 'http://www.wikidata.org/entity/Q976728'), ('filmTitle', 'Halloween 5: The Revenge of Michael Myers'), ('cost', '3000000')]
[('film', 'http://www.wikidata.org/entity/Q1138055'), ('filmTitle', 'Halloween: The Curse of Michael Myers'), ('cost', '5000000')]


6

In [60]:
# Return the cost of films of Friday the 13th franchise
queryString = """
SELECT DISTINCT ?film ?filmTitle ?cost
WHERE { 

    ?film wdt:P179 wd:Q1138878.
    ?film wdt:P1476 ?filmTitle.

    ?film ?property ?cost.
    FILTER(?property = wdt:P2130 || ?property = wdt:P2769)
    
    
} 
ORDER BY ?propertyName
"""

print("Results")
run_query(queryString)

Results
[('film', 'http://www.wikidata.org/entity/Q1243009'), ('filmTitle', 'Friday the 13th Part 2'), ('cost', '1250000')]
[('film', 'http://www.wikidata.org/entity/Q1243013'), ('filmTitle', 'Friday the 13th Part III'), ('cost', '2250000')]
[('film', 'http://www.wikidata.org/entity/Q1454798'), ('filmTitle', 'Friday the 13th Part VII: The New Blood'), ('cost', '2800000')]
[('film', 'http://www.wikidata.org/entity/Q1454799'), ('filmTitle', 'Friday the 13th Part VI: Jason Lives'), ('cost', '3000000')]
[('film', 'http://www.wikidata.org/entity/Q1243019'), ('filmTitle', 'Friday the 13th: The Final Chapter'), ('cost', '2600000')]


5

In [67]:
# Investigate which franchise costed more
queryString = """
SELECT ?franchiseName ?totalCost
WHERE {

    {
        SELECT DISTINCT ?franchiseName (SUM(?cost) AS ?totalCost)
        WHERE { 

            ?film wdt:P179 wd:Q1364022.
            ?film wdt:P1476 ?filmTitle.
            wd:Q1364022 <http://schema.org/name> ?franchiseName.

            ?film ?property ?cost.
            FILTER(?property = wdt:P2130 || ?property = wdt:P2769)
        } 
        GROUP BY ?franchiseName
    }
    UNION
    {
        SELECT DISTINCT ?franchiseName (SUM(?cost) AS ?totalCost)
        WHERE { 

            ?film wdt:P179 wd:Q1138878.
            ?film wdt:P1476 ?filmTitle.
            wd:Q1138878 <http://schema.org/name> ?franchiseName.

            ?film ?property ?cost.
            FILTER(?property = wdt:P2130 || ?property = wdt:P2769)
        } 
        GROUP BY ?franchiseName
    }

}



"""

print("Results")
run_query(queryString)

Results
[('franchiseName', 'Halloween'), ('totalCost', '40825000')]
[('franchiseName', 'Friday the 13th'), ('totalCost', '11900000')]


2

### Point 7 Finish
DESCRIPTION: I started directly with a relation found above, wdt:P2130 “cost” for the Halloween film. Then I returned the cost of all films in Halloween franchise, but observing that only one film has that relation. 
So I checked again the outgoing properties from Halloween franchise’s films and I discover the relation “budget” wdt:P2769.
With both relations I can return the information for some of the film in the franchise.
At the end I was able to find only these two relations, so for every franchise I summed the cost or budget of every film.

RESULT: The result shows that Halloween franchise was the most expensive one. These amounts are not complete, in fact I was able to find some sort of price only for some films.